In [106]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
import os
from tsetlin import TsetlinMachine
import torch
import random
import math
import copy

from itertools import combinations, chain
from collections import deque, defaultdict

DATASET_DIR = '../datasets/'
DATA_FILE = 'bit_2.txt'

text_rows = open(f'{DATASET_DIR}{DATA_FILE}', 'r').read().splitlines()
dataset = [ [int(num) for num in row.split(',')] for row in text_rows]
tensor_dataset = torch.tensor(dataset)
train_x = tensor_dataset[:, :-1]
train_y = tensor_dataset[:, -1]

for _ in range(1000):
    SEED = None
    if SEED:
        random.seed(SEED)
        torch.manual_seed(SEED)
    else:
        seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
        random.seed(seed)
        torch.manual_seed(seed)
        # print(seed)

    tm = TsetlinMachine(train_x.shape[1], 30)
    tm.forward(train_x[:60])

    SEED = None
    if SEED:
        random.seed(SEED)
        torch.manual_seed(SEED)
    else:
        seed = int.from_bytes(os.urandom(8), byteorder="big", signed=False)
        random.seed(seed)
        torch.manual_seed(seed)
        # print(seed)

    tm.l1.W_confidence = torch.randint_like(tm.l1.W, 0, 1000)
    tm.l1.update(tm.l1.out)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
